In [17]:
# https://stackoverflow.com/questions/3579568/choosing-a-file-in-python-with-simple-dialog

from tkinter import Tk     # from tkinter import Tk for Python 3.x
from tkinter.filedialog import askopenfilename

Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
filename = askopenfilename() # show an "Open" dialog box and return the path to the selected file
print(filename)

C:/Users/paste/OneDrive/바탕 화면/CSmatS/B-side/adv_protocol_three.log


In [18]:
# from my Advent of Code 2024 codes

# 파일을 읽기 모드로 열기
file = open(filename, "r")

# 파일 내용 읽기
content = file.read()

# 파일 닫기
file.close()

# 읽은 내용 출력
print(content)

content_list = list(map(str, content.split("\n")))

print(content_list)
len(content_list)

{"opcode": 0, "type": "DH"}
{"opcode": 1, "type": "DH", "public": 285, "parameter": {"p": 401, "g": 347}}
{"opcode": 1, "type": "DH", "public": 229}
{"opcode": 2, "type": "AES", "encryption": "XNymQOrj5AYfV47sDlqf6w=="}
{"opcode": 2, "type": "AES", "encryption": "8vbNPdLVl+VGgDL0aknz3w=="}
['{"opcode": 0, "type": "DH"}', '{"opcode": 1, "type": "DH", "public": 285, "parameter": {"p": 401, "g": 347}}', '{"opcode": 1, "type": "DH", "public": 229}', '{"opcode": 2, "type": "AES", "encryption": "XNymQOrj5AYfV47sDlqf6w=="}', '{"opcode": 2, "type": "AES", "encryption": "8vbNPdLVl+VGgDL0aknz3w=="}']


5

In [19]:
# 아래 내용 실행 중 오류 발생 시 아래 값을 바꿈
error_location = -1
error = None

In [16]:
first_line = eval(content_list[0]) # str -> dict

kie = first_line.keys()

print(kie)
for k in kie:
    print(k)

dict_keys(['opcode', 'type'])
opcode
type


In [20]:
print(content_list)

for i in range(0, len(content_list)):
    print(i)
    print(eval(eval(f"content_list[{i}]")))

['{"opcode": 0, "type": "DH"}', '{"opcode": 1, "type": "DH", "public": 285, "parameter": {"p": 401, "g": 347}}', '{"opcode": 1, "type": "DH", "public": 229}', '{"opcode": 2, "type": "AES", "encryption": "XNymQOrj5AYfV47sDlqf6w=="}', '{"opcode": 2, "type": "AES", "encryption": "8vbNPdLVl+VGgDL0aknz3w=="}']
0
{'opcode': 0, 'type': 'DH'}
1
{'opcode': 1, 'type': 'DH', 'public': 285, 'parameter': {'p': 401, 'g': 347}}
2
{'opcode': 1, 'type': 'DH', 'public': 229}
3
{'opcode': 2, 'type': 'AES', 'encryption': 'XNymQOrj5AYfV47sDlqf6w=='}
4
{'opcode': 2, 'type': 'AES', 'encryption': '8vbNPdLVl+VGgDL0aknz3w=='}


In [29]:
def line_validity_check(target_list: list, line_number: int, opcode: int, typ: str, additional_keys: list = []): # type is a function so could not use exact word
    error_list = []
    target = eval(target_list[line_number])
    
    keys = target.keys()
    
    if "opcode" in keys:
        if target["opcode"] != opcode:
            error_list.append(f"unpredicted opcode (!= {opcode})")
    else:
        error_list.append("missing opcode")
    if "type" in keys:
        if target["type"] != typ:
            error_list.append(f"unpredicted type (!= {typ})")
    else:
        error_list.append("missing type")
    
    for key in additional_keys:
        if key not in keys:
            error_list.append(f"missing {key}")
    
    # no need to handle when keys not mentioned in additional_keys are in target, ig
    
    return error_list

In [ ]:
def json_validity_check(target: list):
    if len(target) < 4:
        print("incomplete json that does not contain enough information to decode messages")
        return False
    
    l0 = line_validity_check(target, 0, 0, "RSA")
    l1 = line_validity_check(target, 1, 1, "RSA", ["public", "parameter"])
    l2 = line_validity_check(target, 2, 2, "RSA", ["encrypted_key"])
    for i in range(3, len(target)): # handling multiple messages
        exec(f"l{i} = line_validity_check(target, {i}, 2, \"AES\", [\"encryption\"])")
    
    turnagain = True
    for i in range(len(target)):
        current = eval(f"l{i}")
        if current != []:
            print(f"error in line {i}\n{target[i]}\n{str(current)[1:-1].replace('\'', '')}") # removing bracket and single quote (why did I waste time here)
            turnagain = False
    return turnagain # re-turn haha I'm funny...

In [31]:
json_validity_check(content_list)

error in line 0
{"opcode": 0, "type": "DH"}
unpredicted type (!= RSA)
error in line 1
{"opcode": 1, "type": "DH", "public": 285, "parameter": {"p": 401, "g": 347}}
unpredicted type (!= RSA)
error in line 2
{"opcode": 1, "type": "DH", "public": 229}
unpredicted opcode (!= 2), unpredicted type (!= RSA), missing encrypted_key


False